# Boolean Extended IR Model

## Document Representation
I've first represented the documents as binary vectors, where each term's presence is indicated by 1 and absence by 0.

## Boolean Query Processing
I've implemented basic query processing that supports AND, OR, and NOT. The query processor will take a Boolean query, parse it, and apply the operations on the term-document matrix.

## Ranking Algorithm
The ranking algorithm will rank documents based on the number of matching terms. For Boolean expressions, the rank will be binary (either the document matches or it doesn’t). For more complex queries, partial satisfaction can be taken into account.

In [7]:
from typing import List, Set, Tuple

class BooleanExtendedIRModel:
    def __init__(self, documents: List[str]):
        """
        Initialize the Boolean Extended IR Model
        
        Args:
            documents (List[str]): Collection of input documents
        """
        self.documents = documents
        self.vocab: Set[str] = set()
        self.processed_docs: List[List[str]] = []
        self.term_doc_matrix: List[List[int]] = []
        
        # Term weight matrix (optional enhancement)
        self.term_weights: dict = {}
        
        # Preprocessing
        self.preprocess_documents()
    
    def preprocess_documents(self) -> None:
        """
        Preprocess documents into vocabulary and processed documents
        """
        
        for doc in self.documents:
            # Tokenize and clean terms
            terms = self._clean_and_tokenize(doc)
            self.vocab.update(terms)
            self.processed_docs.append(terms)
        
        # Convert vocab to sorted list for consistent indexing
        self.vocab = sorted(list(self.vocab))

        print("Vocab:", self.vocab)
        
        # Create term document matrix
        self.term_doc_matrix = self.create_term_document_matrix()

        for i, row in enumerate(self.term_doc_matrix):
            print(f"Document {i + 1}: {row}")
        
        # Initialize term weights (optional)
        self._initialize_term_weights()
    
    def _clean_and_tokenize(self, document: str) -> List[str]:
        """
        Clean and tokenize a document
        
        Args:
            document (str): Input document string
        
        Returns:
            List[str]: Cleaned and tokenized terms
        """
        # Basic cleaning: lowercase, split
        return document.lower().split()
    
    def _initialize_term_weights(self) -> None:
        """
        Initialize term weights based on term frequency
        """
        for term in self.vocab:
            # Simple weight calculation based on document frequency
            doc_frequency = sum(self.get_term_vector(term))
            self.term_weights[term] = 1 + (doc_frequency / len(self.documents))
    
    def create_term_document_matrix(self) -> List[List[int]]:
        """
        Create binary term-document matrix
        
        Returns:
            List[List[int]]: Binary matrix representing term presence
        """
        matrix = []
        for doc in self.processed_docs:
            vector = [1 if term in doc else 0 for term in self.vocab]
            matrix.append(vector)
        
        return matrix
    
    def get_term_vector(self, term: str) -> List[int]:
        """
        Get binary vector for a given term
        
        Args:
            term (str): Term to retrieve vector for
        
        Returns:
            List[int]: Binary vector representing term presence
        """
        if term not in self.vocab:
            # logger.warning(f"Term '{term}' not in vocabulary")
            return [0] * len(self.documents)
        
        index = self.vocab.index(term)
        return [doc[index] for doc in self.term_doc_matrix]
    
    def and_operation(self, vec1: List[int], vec2: List[int]) -> List[int]:
        """
        Perform AND operation on two vectors
        
        Args:
            vec1 (List[int]): First binary vector
            vec2 (List[int]): Second binary vector
        
        Returns:
            List[int]: Resulting binary vector
        """
        return [x & y for x, y in zip(vec1, vec2)]
    
    def or_operation(self, vec1: List[int], vec2: List[int]) -> List[int]:
        """
        Perform OR operation on two vectors
        
        Args:
            vec1 (List[int]): First binary vector
            vec2 (List[int]): Second binary vector
        
        Returns:
            List[int]: Resulting binary vector
        """
        return [x | y for x, y in zip(vec1, vec2)]
    
    def not_operation(self, vec: List[int]) -> List[int]:
        """
        Perform NOT operation on a vector
        
        Args:
            vec (List[int]): Input binary vector
        
        Returns:
            List[int]: Inverted binary vector
        """
        return [1 - x for x in vec]
    
    def process_query(self, query: str) -> List[int]:
        """
        Process complex Boolean query
        
        Args:
            query (str): Boolean query string
        
        Returns:
            List[int]: Result vector indicating matching documents
        """
        
        terms = query.split()
        result_vector = []

        i = 0
        while i < len(terms):
            term = terms[i]
            
            if term == "AND":
                next_term = terms[i + 1]
                result_vector = self.and_operation(result_vector, self.get_term_vector(next_term))
                i += 2
            elif term == "OR":
                next_term = terms[i + 1]
                result_vector = self.or_operation(result_vector, self.get_term_vector(next_term))
                i += 2
            elif term == "NOT":
                next_term = terms[i + 1]
                result_vector = self.not_operation(self.get_term_vector(next_term))
                i += 2
            else:
                result_vector = self.get_term_vector(term)
                i += 1

        return result_vector
    
    def rank_documents(self, result_vector: List[int]) -> List[Tuple[int, str]]:
        """
        Rank documents based on result vector
        
        Args:
            result_vector (List[int]): Binary vector of matching documents
        
        Returns:
            List[Tuple[int, str]]: List of ranked documents with indices
        """
        ranked_docs = [
            (i, self.documents[i]) 
            for i in range(len(result_vector)) 
            if result_vector[i] == 1
        ]
        return ranked_docs


def main():
    # Sample documents
    documents = [
        "algorithm optimization sorting",
        "algorithm dynamic graph theory",
        "optimization dynamic programming"
    ]
    
    # Create Boolean Extended IR Model instance
    ir_model = BooleanExtendedIRModel(documents)
    
    # Example queries
    queries = [
        "algorithm AND optimization NOT dynamic",
        "optimization OR dynamic",
        "algorithm NOT graph",
        "algorithm NOT graph AND optimization OR dynamic",
    ]
    
    # Process and rank documents for each query
    for query in queries:
        print(f"\nQuery: {query}")
        
        result_vector = ir_model.process_query(query)
        ranked_docs = ir_model.rank_documents(result_vector)
        
        print("Ranked Documents:")
        for doc in ranked_docs:
            print(f"Document {doc[0] + 1}: {doc[1]}")

if __name__ == "__main__":
    main()
    

Vocab: ['algorithm', 'dynamic', 'graph', 'optimization', 'programming', 'sorting', 'theory']
Document 1: [1, 0, 0, 1, 0, 1, 0]
Document 2: [1, 1, 1, 0, 0, 0, 1]
Document 3: [0, 1, 0, 1, 1, 0, 0]

Query: algorithm AND optimization NOT dynamic
Ranked Documents:
Document 1: algorithm optimization sorting

Query: optimization OR dynamic
Ranked Documents:
Document 1: algorithm optimization sorting
Document 2: algorithm dynamic graph theory
Document 3: optimization dynamic programming

Query: algorithm NOT graph
Ranked Documents:
Document 1: algorithm optimization sorting
Document 3: optimization dynamic programming

Query: algorithm NOT graph AND optimization OR dynamic
Ranked Documents:
Document 1: algorithm optimization sorting
Document 2: algorithm dynamic graph theory
Document 3: optimization dynamic programming
